## Let's use hypothesis testing to see if data have the same mean
Recall, p-value is the probability of observing your data (or more extrem) given that the null hypothesis is true.

In our case:
**Null hypothesis** is that means are equal
**Alternative hyp.** is that they differ

If p-value < 5%, it means that it is quite unlikely to oberve the provided data under the assumption that the null hypothesis is true. Therefore, we **can** reject it in favor of the alternative hypothesis.


In [ ]:
# prepare the filenames for proudction router


days = [22, 23, 24]
production_files = ["/home/durkota/data/projects/tacr/data/Experiment-8/omnia1/Traffic/2017-07-"+str(day)+"-147.32.83.179.binetflow" for day in days]
hp_files = ["/home/durkota/data/projects/tacr/data/Experiment-8/omnia2/Traffic/2017-07-"+str(day)+"-147.32.83.175.binetflow" for day in days]

port = '80'


In [ ]:
#!/usr/bin/env python3.6 -u
# Author: Karel Durkota
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import flowExtractor 


In [ ]:
# get sequence of unique # of IP per hour 
def getHour(string):
    return string.split(" ")[1].split(":")[0]

def getSequenceFromData(data, keys):
    ser = pd.Series(data[1:,0]).groupby(list(map(getHour, data[1:,1]))).count()
    seq = []
    for key in keys:
        if str(key) in ser.index:
            seq.append(ser.get(str(key)))
        else:
            seq.append(0)
    return seq


In [ ]:
col = dict()

def getFullSequence(filenames, port):
    global col
    seq = []
    for filename in filenames:
        data = np.array(flowExtractor.extract(['-p', port, '-f', filename]))
        
        header = data[0,:]
        col = dict([(header[i], i) for i in range(len(header))])
        
        keys = range(0,24)
        seq.append(getSequenceFromData(data[:,[col['SrcAddr'],col['StartTime']]], keys))
    return [y for x in seq for y in x]

#production = ["/home/durkota/data/projects/tacr/data/Experiment-8/omnia1/Traffic/2017-07-"+str(day)+"-147.32.83.179.binetflow" for day in days]
#print(getFullSequence(production, '80'))


In [ ]:
              
              
# getting full sequences
seq1 = getFullSequence(production_files, port)
seq2 = getFullSequence(hp_files, port)

# run hypothesis test
# Hypothesis testing


print("Mean1 = {0}".format(np.mean(seq1)))
print("Count = {0}".format(len(seq1)))
print("Mean2 = {0}".format(np.mean(seq2)))
print("Count = {0}".format(len(seq2)))

test = stats.ttest_ind(a= seq1, b= seq2, equal_var=False)  # Pop mean
print(test)

# plot histograms


bins = np.linspace(0,10,20)
plt.hist(seq1, bins, alpha=0.5)
plt.hist(seq2, bins, alpha=0.5)
plt.show()